## EVA / Clinvar Comparison

Compares ClinVar record accessions in EVA OTP evidence strings to raw ClinVar downloads. 

In [1]:
from dotenv import load_dotenv; load_dotenv()
from data_source import catalog
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from IPython.display import JSON
import pandas as pd
import json
spark = SparkSession.builder.getOrCreate()
cat = catalog.load()

In [25]:
from IPython.display import JSON
def pjson(df, **kwargs):
    return JSON(json.loads(df.toJSON().take(1)[0]), **kwargs)

### EVA

In [9]:
path = cat.download('otpev', 'eva', version='v20.06')
path

'/tmp/data_source_cache/otpev/eva/v20.06/20200616T000000/data.parquet'

In [60]:
df = spark.read.parquet(path)
df.printSchema()

root
 |-- access_level: string (nullable = true)
 |-- disease: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- source_name: string (nullable = true)
 |-- evidence: struct (nullable = true)
 |    |-- clinical_significance: string (nullable = true)
 |    |-- date_asserted: string (nullable = true)
 |    |-- evidence_codes: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- gene2variant: struct (nullable = true)
 |    |    |-- date_asserted: string (nullable = true)
 |    |    |-- evidence_codes: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- functional_consequence: string (nullable = true)
 |    |    |-- is_associated: boolean (nullable = true)
 |    |    |-- provenance_type: struct (nullable = true)
 |    |    |    |-- database: struct (nullable = true)
 |    |    |    |    |-- dbxref: struct (nullable = true)
 |    |    |    |    |    

In [38]:
pjson(df, expanded=True)

<IPython.core.display.JSON object>

In [61]:
df.groupBy('evidence.variant2disease.clinical_significance').count().show()

+---------------------+-----+
|clinical_significance|count|
+---------------------+-----+
|                 null| 8232|
|              Affects|  112|
|          association|  153|
|           protective|   30|
|        drug response| 4595|
|    Likely pathogenic|30683|
|           Pathogenic|81333|
+---------------------+-----+



In [67]:
csigs = df.select('evidence.variant2disease.clinical_significance').dropDuplicates().toPandas().iloc[:,0].str.lower()
csigs

0                 None
1              affects
2          association
3           protective
4        drug response
5    likely pathogenic
6           pathogenic
Name: clinical_significance, dtype: object

In [52]:
def get_rcvs(df, path):
    return (
        df
        .filter(F.col(path + '.id') == 'http://identifiers.org/clinvar')
        .withColumn('id', F.element_at(F.split(F.col(path + '.url'), '/'), -1))
        .select('id')
        .toPandas().iloc[:,0].drop_duplicates()
    )
eva_rcvs_1 = get_rcvs(df, 'evidence.gene2variant.provenance_type.database.dbxref')
eva_rcvs_2 = get_rcvs(df, 'evidence.variant2disease.provenance_type.database.dbxref')

In [53]:
eva_rcvs_1

0         RCV000007792
1         RCV000186552
2         RCV000009594
3         RCV000133591
4         RCV000200158
              ...     
116900    RCV000736266
116901    RCV000736267
116902    RCV000736269
116903    RCV000815476
116904    RCV000010376
Name: id, Length: 99856, dtype: object

### ClinVar

In [27]:
# Download variant_summary, not submission_summary since this contains submission records
# with "SCV" accessions as opposed to "RCV" accessions for top-level records
!wget -P /tmp https://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/archive/variant_summary_2020-06.txt.gz

--2020-06-19 13:07:51--  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/archive/variant_summary_2020-06.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 2607:f220:41e:250::10
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56326157 (54M) [application/x-gzip]
Saving to: ‘/tmp/variant_summary_2020-06.txt.gz.1’

variant_summary_202 100%[===================>]  53.72M  6.93MB/s    in 9.8s    

2020-06-19 13:08:01 (5.49 MB/s) - ‘/tmp/variant_summary_2020-06.txt.gz.1’ saved [56326157/56326157]



In [82]:
dfvs = pd.read_csv('/tmp/variant_summary_2020-06.txt.gz', sep='\t')
dfvs.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID
0,633632,single nucleotide variant,NM_000112.3(SLC26A2):c.892C>A (p.His298Asn),1836,SLC26A2,HGNC:10994,Uncertain significance,0,"Dec 14, 2018",-1,...,C,A,5q32,"criteria provided, single submitter",1,NaN,N,-,2,651753
1,633632,single nucleotide variant,NM_000112.3(SLC26A2):c.892C>A (p.His298Asn),1836,SLC26A2,HGNC:10994,Uncertain significance,0,"Dec 14, 2018",-1,...,C,A,5q32,"criteria provided, single submitter",1,NaN,N,-,2,651753
2,633633,single nucleotide variant,NM_000112.3(SLC26A2):c.1234G>A (p.Val412Ile),1836,SLC26A2,HGNC:10994,Likely benign,0,"Dec 24, 2019",-1,...,G,A,5q32,"criteria provided, single submitter",1,NaN,N,-,2,650535
3,633633,single nucleotide variant,NM_000112.3(SLC26A2):c.1234G>A (p.Val412Ile),1836,SLC26A2,HGNC:10994,Likely benign,0,"Dec 24, 2019",-1,...,G,A,5q32,"criteria provided, single submitter",1,NaN,N,-,2,650535
4,633634,single nucleotide variant,NM_000112.3(SLC26A2):c.2164T>C (p.Ser722Pro),1836,SLC26A2,HGNC:10994,Uncertain significance,0,"Sep 12, 2018",-1,...,T,C,5q32,"criteria provided, single submitter",1,NaN,N,-,2,651134


In [48]:
dfvs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539560 entries, 0 to 1539559
Data columns (total 31 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   #AlleleID             1539560 non-null  int64 
 1   Type                  1539560 non-null  object
 2   Name                  1537466 non-null  object
 3   GeneID                1539560 non-null  int64 
 4   GeneSymbol            1539560 non-null  object
 5   HGNC_ID               1539560 non-null  object
 6   ClinicalSignificance  1539560 non-null  object
 7   ClinSigSimple         1539560 non-null  int64 
 8   LastEvaluated         1539560 non-null  object
 9   RS# (dbSNP)           1539560 non-null  int64 
 10  nsv/esv (dbVar)       1539560 non-null  object
 11  RCVaccession          1539560 non-null  object
 12  PhenotypeIDS          1539560 non-null  object
 13  PhenotypeList         1539560 non-null  object
 14  Origin                1539560 non-null  object
 15

In [69]:
csigs

0                 None
1              affects
2          association
3           protective
4        drug response
5    likely pathogenic
6           pathogenic
Name: clinical_significance, dtype: object

In [70]:
dfvs['ClinicalSignificance'].value_counts().sort_values().tail(25)

Pathogenic, Affects                                              25
Benign, other                                                    25
Likely benign, other                                             28
Conflicting interpretations of pathogenicity, risk factor        30
Conflicting interpretations of pathogenicity, other              36
Pathogenic, drug response                                        53
Pathogenic, risk factor                                          79
protective                                                       84
Pathogenic, other                                               202
Affects                                                         270
conflicting data from submitters                                454
association                                                     472
risk factor                                                     987
no interpretation for the single variant                       1222
drug response                                   

In [71]:
def accept_cs(cs):
    if 'conflicting' in cs:
        return False
    if 'pathogenic' in cs:
        return True
    if 'protective' in cs:
        return True
    if cs in csigs:
        return True
    return False
    
dfvs['accept'] = dfvs['ClinicalSignificance'].str.lower().apply(accept_cs)
dfvs['accept'].value_counts()

False    1277510
True      262050
Name: accept, dtype: int64

In [73]:
dfvs[dfvs['accept']]['ClinicalSignificance'].value_counts()

Pathogenic                                     177449
Likely pathogenic                               70985
Pathogenic/Likely pathogenic                    13046
Pathogenic, other                                 202
protective                                         84
Pathogenic, risk factor                            79
Pathogenic, drug response                          53
Pathogenic, Affects                                25
Likely pathogenic, risk factor                     23
Pathogenic/Likely pathogenic, drug response        18
Likely pathogenic, drug response                   16
Pathogenic/Likely pathogenic, other                14
Pathogenic, protective                             12
Pathogenic/Likely pathogenic, risk factor          12
protective, risk factor                             9
Pathogenic, association                             6
Likely pathogenic, other                            4
Pathogenic, drug response, other                    3
Likely pathogenic, associati

In [49]:
dfvs['RCVaccession'].head()

0    RCV000807178;RCV000807178;RCV000807178;RCV0008...
1    RCV000807178;RCV000807178;RCV000807178;RCV0008...
2    RCV000805701;RCV000805701;RCV000805701;RCV0008...
3    RCV000805701;RCV000805701;RCV000805701;RCV0008...
4    RCV000806427;RCV000806427;RCV000806427;RCV0008...
Name: RCVaccession, dtype: object

In [74]:
clinvar_rcvs = pd.Series([
    v
    for rv in dfvs[dfvs['accept']]['RCVaccession']
    for v in rv.split(';')
]).drop_duplicates()
clinvar_rcvs

0         RCV000799345
2         RCV000798536
4         RCV000809513
6         RCV000815237
8         RCV000818305
              ...     
378757    RCV001171373
378758    RCV001171376
378759    RCV001171377
378760    RCV001171379
378762    RCV001171381
Length: 179381, dtype: object

In [75]:
len(eva_rcvs_1), len(eva_rcvs_2), len(clinvar_rcvs)

(99856, 99856, 179381)

In [76]:
len(set(eva_rcvs_1) & set(eva_rcvs_2))

99856

In [77]:
len(set(eva_rcvs_1) - set(clinvar_rcvs))

7339

In [79]:
len(set(clinvar_rcvs) - set(eva_rcvs_1))

86864